## 1. Environment Setup

In [ ]:
# Check GPU availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime > Change runtime type > GPU")

In [ ]:
# Install required packages - using Colab-compatible versions
# We avoid facenet-pytorch due to numpy version conflicts

!pip install -q kaggle wandb
!pip install -q albumentations

# Install MTCNN separately (face detection)
!pip install -q mtcnn

print("✅ Packages installed! Restarting runtime...")

# Auto-restart to ensure clean state
import os
os.kill(os.getpid(), 9)

In [ ]:
# Core imports
import os
import json
import numpy as np
from pathlib import Path
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torchvision.models as models
from torchvision import transforms

# Face Detection (MTCNN uses TensorFlow backend in this version)
from mtcnn import MTCNN

# Image Processing
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Visualization
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Set seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ All imports successful!")
print(f"📍 Using device: {DEVICE}")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 PyTorch version: {torch.__version__}")

## 2. Kaggle Setup & Data Download

In [ ]:
# Setup Kaggle credentials from the kaggle.json file in the notebooks folder
# This file is bundled with the project - no need to manually enter credentials
import os
import json

# Kaggle credentials (from notebooks/kaggle.json)
KAGGLE_USERNAME = "zyadelfeki1"
KAGGLE_KEY = "0ca3cf05892a2d79ecc9fe7f6ae0d05e"

# Create Kaggle config directory and file
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
kaggle_config = {"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}

with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_config, f)

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

print("✅ Kaggle credentials configured automatically!")

In [ ]:
# Download dataset directly from Kaggle to Colab (no local download needed)
# The dataset stays on Colab's cloud storage, not your local machine

# Option 1: Labeled Faces in the Wild (LFW) - Classic benchmark
# !kaggle datasets download -d jessicali9530/lfw-dataset -p ./data --unzip

# Option 2: Celebrity Face Dataset - Good for recognition training
# !kaggle datasets download -d vishesh1412/celebrity-face-image-dataset -p ./data --unzip

# Option 3: FFHQ Face Dataset (default) - High quality faces
!kaggle datasets download -d greatgamedota/ffhq-face-data-set -p ./data --unzip

# The -p flag downloads directly to ./data
# The --unzip flag extracts automatically (no separate unzip step)
# Data stays in Colab's ephemeral storage, never downloaded to your local machine

print("\n📁 Dataset downloaded to Colab cloud storage!")
print("(Data is NOT on your local machine - it's in Colab's temporary storage)")
!ls -la ./data

In [ ]:
# Alternative: Use a custom dataset structure
# Expected structure:
# data/
#   person_1/
#     img1.jpg
#     img2.jpg
#   person_2/
#     img1.jpg
#     ...

DATA_DIR = Path("./data")

def explore_dataset(data_dir: Path) -> List[Dict]:
    """Explore dataset structure and create a manifest."""
    from collections import Counter
    records = []
    
    for person_dir in data_dir.iterdir():
        if person_dir.is_dir():
            person_name = person_dir.name
            images = list(person_dir.glob("*.jpg")) + list(person_dir.glob("*.png")) + list(person_dir.glob("*.jpeg"))
            for img_path in images:
                records.append({
                    "person": person_name,
                    "image_path": str(img_path),
                    "image_name": img_path.name
                })
    
    # Statistics
    person_counts = Counter(r['person'] for r in records)
    counts = list(person_counts.values())
    
    print(f"📊 Dataset Summary:")
    print(f"   Total images: {len(records)}")
    print(f"   Total persons: {len(person_counts)}")
    print(f"   Images per person - min: {min(counts)}, max: {max(counts)}, avg: {sum(counts)/len(counts):.1f}")
    
    return records

# Explore the dataset
df_dataset = explore_dataset(DATA_DIR)

## 3. Data Preprocessing & Face Detection

In [ ]:
# Initialize MTCNN for face detection (TensorFlow-based)
mtcnn = MTCNN()

# Face preprocessing config
FACE_SIZE = 160
FACE_MARGIN = 20

print("✅ MTCNN initialized for face detection")

In [ ]:
def detect_and_align_face(image_path: str, detector: MTCNN, face_size: int = 160, margin: int = 20) -> Optional[np.ndarray]:
    """
    Detect and align face from image using MTCNN.
    Returns face image as numpy array (face_size x face_size x 3) or None if no face detected.
    """
    try:
        # Load image
        img = cv2.imread(image_path)
        if img is None:
            return None
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        detections = detector.detect_faces(img_rgb)
        
        if not detections:
            return None
        
        # Get the largest face
        largest_face = max(detections, key=lambda x: x['box'][2] * x['box'][3])
        
        # Extract bounding box with margin
        x, y, w, h = largest_face['box']
        x = max(0, x - margin)
        y = max(0, y - margin)
        w = w + 2 * margin
        h = h + 2 * margin
        
        # Crop and resize face
        face = img_rgb[y:y+h, x:x+w]
        if face.size == 0:
            return None
            
        face = cv2.resize(face, (face_size, face_size))
        
        return face
        
    except Exception as e:
        return None


def preprocess_dataset(records: List[Dict], detector: MTCNN, output_dir: Path) -> List[Dict]:
    """Detect faces and save aligned face crops."""
    output_dir.mkdir(parents=True, exist_ok=True)
    
    valid_records = []
    
    for record in tqdm(records, desc="Processing faces"):
        face = detect_and_align_face(record['image_path'], detector)
        
        if face is not None:
            # Save aligned face
            person_dir = output_dir / record['person']
            person_dir.mkdir(exist_ok=True)
            
            face_path = person_dir / record['image_name']
            
            # Save as RGB image
            Image.fromarray(face).save(face_path)
            
            valid_records.append({
                "person": record['person'],
                "face_path": str(face_path)
            })
    
    return valid_records

# Process dataset
PROCESSED_DIR = Path("./processed_faces")
face_records = preprocess_dataset(df_dataset, mtcnn, PROCESSED_DIR)

# Use the list directly
df_faces = face_records

print(f"\n✅ Processed {len(face_records)} faces from {len(set(r['person'] for r in face_records))} persons")

## 4. Custom Dataset & DataLoader

In [ ]:
class FaceDataset(Dataset):
    """Custom dataset for face recognition training."""
    
    def __init__(
        self,
        records: List[Dict],
        transform=None,
        label_encoder=None
    ):
        self.records = records
        self.transform = transform
        
        persons = [r['person'] for r in records]
        
        if label_encoder is None:
            self.label_encoder = LabelEncoder()
            self.labels = self.label_encoder.fit_transform(persons)
        else:
            self.label_encoder = label_encoder
            self.labels = self.label_encoder.transform(persons)
        
        self.num_classes = len(self.label_encoder.classes_)
    
    def __len__(self):
        return len(self.records)
    
    def __getitem__(self, idx):
        record = self.records[idx]
        
        # Load image
        img = cv2.imread(record['face_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        else:
            img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        
        label = self.labels[idx]
        
        return img, label


# Data Augmentation
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.GaussNoise(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.3),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.3),
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2()
])

# Split dataset
persons = [r['person'] for r in df_faces]
train_records, val_records = train_test_split(
    df_faces,
    test_size=0.2,
    stratify=persons,
    random_state=SEED
)

print(f"Train: {len(train_records)}, Val: {len(val_records)}")

# Create datasets
train_dataset = FaceDataset(train_records, transform=train_transform)
val_dataset = FaceDataset(val_records, transform=val_transform, label_encoder=train_dataset.label_encoder)

# Create dataloaders
BATCH_SIZE = 32
NUM_WORKERS = 2

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"\n✅ DataLoaders ready!")
print(f"   Number of classes: {train_dataset.num_classes}")

## 5. Model Architecture

In [ ]:
class FaceRecognitionModel(nn.Module):
    """
    Face recognition model using ResNet50 backbone with custom embedding head.
    
    Architecture:
    - Backbone: ResNet50 (pretrained on ImageNet, fine-tuned for faces)
    - Head: ArcFace-style classifier for better angular margin
    """
    
    def __init__(
        self,
        num_classes: int,
        embedding_size: int = 512,
        dropout: float = 0.3
    ):
        super().__init__()
        
        # ResNet50 backbone (pretrained on ImageNet)
        backbone = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        
        # Remove the final FC layer
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        backbone_out_features = 2048
        
        # Freeze early layers for transfer learning
        for name, param in self.backbone.named_parameters():
            # Unfreeze layer3 and layer4 only
            if 'layer3' not in name and 'layer4' not in name:
                param.requires_grad = False
        
        # Custom embedding head
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout)
        self.embedding_layer = nn.Linear(backbone_out_features, embedding_size)
        self.bn = nn.BatchNorm1d(embedding_size)
        
        # Classifier (for training with ArcFace-style loss)
        self.classifier = nn.Linear(embedding_size, num_classes)
        
        self.embedding_size = embedding_size
        self.num_classes = num_classes
    
    def forward(self, x, return_embeddings: bool = False):
        # Backbone features
        features = self.backbone(x)
        features = self.flatten(features)
        
        # Embedding
        features = self.dropout(features)
        embeddings = self.embedding_layer(features)
        embeddings = self.bn(embeddings)
        embeddings = F.normalize(embeddings, p=2, dim=1)
        
        if return_embeddings:
            return embeddings
        
        # Classification
        logits = self.classifier(embeddings)
        
        return logits, embeddings
    
    def get_embedding(self, x):
        """Get normalized face embedding for inference."""
        return self.forward(x, return_embeddings=True)


# Initialize model
model = FaceRecognitionModel(
    num_classes=train_dataset.num_classes,
    embedding_size=512,
    dropout=0.3
).to(DEVICE)

# Print model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📐 Model Summary:")
print(f"   Backbone: ResNet50 (ImageNet pretrained)")
print(f"   Embedding size: 512")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")

## 6. Loss Functions & Optimizer

In [ ]:
class ArcFaceLoss(nn.Module):
    """
    ArcFace loss for better face recognition.
    Adds angular margin to the softmax loss for tighter clusters.
    """
    
    def __init__(self, scale: float = 30.0, margin: float = 0.5):
        super().__init__()
        self.scale = scale
        self.margin = margin
        self.ce = nn.CrossEntropyLoss()
    
    def forward(self, logits, labels):
        # Get cosine values
        cos_theta = logits
        
        # Add margin to target class
        one_hot = F.one_hot(labels, num_classes=logits.size(1)).float()
        
        # Compute arccos, add margin, then cos again
        theta = torch.acos(torch.clamp(cos_theta, -1.0 + 1e-7, 1.0 - 1e-7))
        marginal_cos = torch.cos(theta + self.margin * one_hot)
        
        # Scale and compute loss
        scaled_logits = self.scale * marginal_cos
        loss = self.ce(scaled_logits, labels)
        
        return loss


class CombinedLoss(nn.Module):
    """Combined CrossEntropy + Triplet Loss for robust training."""
    
    def __init__(self, margin: float = 0.3, ce_weight: float = 0.5):
        super().__init__()
        self.ce = nn.CrossEntropyLoss()
        self.triplet = nn.TripletMarginLoss(margin=margin)
        self.ce_weight = ce_weight
    
    def forward(self, logits, embeddings, labels):
        ce_loss = self.ce(logits, labels)
        
        # Simple triplet mining within batch
        triplet_loss = self._batch_hard_triplet_loss(embeddings, labels)
        
        return self.ce_weight * ce_loss + (1 - self.ce_weight) * triplet_loss
    
    def _batch_hard_triplet_loss(self, embeddings, labels, margin=0.3):
        """Compute triplet loss with batch hard mining."""
        # Pairwise distances
        dist_matrix = torch.cdist(embeddings, embeddings)
        
        # Create masks
        labels = labels.unsqueeze(1)
        same_label = labels == labels.T
        diff_label = ~same_label
        
        # Hardest positive: max distance within same class
        pos_dist = dist_matrix * same_label.float()
        hardest_pos, _ = pos_dist.max(dim=1)
        
        # Hardest negative: min distance to different class
        neg_dist = dist_matrix + 1e9 * same_label.float()  # Mask same class
        hardest_neg, _ = neg_dist.min(dim=1)
        
        # Triplet loss
        loss = F.relu(hardest_pos - hardest_neg + margin).mean()
        
        return loss


# Setup training
criterion = CombinedLoss(margin=0.3, ce_weight=0.7)

optimizer = AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4,
    weight_decay=1e-4
)

scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=5,
    T_mult=2
)

print("✅ Loss and optimizer configured!")

## 7. Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc="Training")
    
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        logits, embeddings = model(images)
        loss = criterion(logits, embeddings, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = logits.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
        pbar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'acc': f"{100.*correct/total:.2f}%"
        })
    
    return total_loss / len(loader), correct / total


@torch.no_grad()
def validate(model, loader, criterion, device):
    """Validate model."""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_embeddings = []
    
    for images, labels in tqdm(loader, desc="Validating"):
        images = images.to(device)
        labels = labels.to(device)
        
        logits, embeddings = model(images)
        loss = criterion(logits, embeddings, labels)
        
        total_loss += loss.item()
        
        _, predicted = logits.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_embeddings.append(embeddings.cpu())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_embeddings = torch.cat(all_embeddings).numpy()
    
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro', zero_division=0
    )
    
    return {
        'loss': total_loss / len(loader),
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'embeddings': all_embeddings,
        'predictions': all_preds,
        'labels': all_labels
    }

In [ ]:
# Training configuration
NUM_EPOCHS = 30
SAVE_DIR = Path("./checkpoints")
SAVE_DIR.mkdir(exist_ok=True)

best_val_acc = 0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("🚀 Starting training...\n")

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 40)
    
    # Train
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, DEVICE
    )
    
    # Validate
    val_results = validate(model, val_loader, criterion, DEVICE)
    
    # Step scheduler
    scheduler.step()
    
    # Log
    print(f"\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    print(f"Val Loss: {val_results['loss']:.4f} | Val Acc: {val_results['accuracy']*100:.2f}%")
    print(f"Val F1: {val_results['f1']:.4f}")
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_results['loss'])
    history['val_acc'].append(val_results['accuracy'])
    
    # Save best model
    if val_results['accuracy'] > best_val_acc:
        best_val_acc = val_results['accuracy']
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_accuracy': best_val_acc,
            'num_classes': model.num_classes,
            'label_encoder_classes': train_dataset.label_encoder.classes_.tolist()
        }, SAVE_DIR / "best_model.pth")
        print(f"💾 Saved best model with accuracy: {best_val_acc*100:.2f}%")

print(f"\n✅ Training complete! Best validation accuracy: {best_val_acc*100:.2f}%")

## 8. Evaluation & Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy
axes[1].plot([a*100 for a in history['train_acc']], label='Train Acc')
axes[1].plot([a*100 for a in history['val_acc']], label='Val Acc')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training & Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(SAVE_DIR / "training_history.png", dpi=150)
plt.show()

In [ ]:
# Confusion Matrix
val_results = validate(model, val_loader, criterion, DEVICE)

cm = confusion_matrix(val_results['labels'], val_results['predictions'])

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=False, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(SAVE_DIR / "confusion_matrix.png", dpi=150)
plt.show()

In [ ]:
# t-SNE Visualization of Embeddings
from sklearn.manifold import TSNE

# Subsample for visualization
n_samples = min(1000, len(val_results['embeddings']))
indices = np.random.choice(len(val_results['embeddings']), n_samples, replace=False)

embeddings_subset = val_results['embeddings'][indices]
labels_subset = val_results['labels'][indices]

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=SEED, perplexity=30)
embeddings_2d = tsne.fit_transform(embeddings_subset)

# Plot
plt.figure(figsize=(12, 10))
scatter = plt.scatter(
    embeddings_2d[:, 0],
    embeddings_2d[:, 1],
    c=labels_subset,
    cmap='tab20',
    alpha=0.7,
    s=30
)
plt.title('t-SNE Visualization of Face Embeddings')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.colorbar(scatter, label='Person ID')
plt.savefig(SAVE_DIR / "tsne_embeddings.png", dpi=150)
plt.show()

## 9. Export Models for Deployment

In [ ]:
# Load best model
checkpoint = torch.load(SAVE_DIR / "best_model.pth")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Loaded best model from epoch {checkpoint['epoch']+1}")
print(f"   Validation accuracy: {checkpoint['val_accuracy']*100:.2f}%")

In [ ]:
# Export to TorchScript for production
class EmbeddingModel(nn.Module):
    """Wrapper for embedding-only inference."""
    
    def __init__(self, full_model):
        super().__init__()
        self.backbone = full_model.backbone
        self.dropout = full_model.dropout
        self.embedding_layer = full_model.embedding_layer
        self.bn = full_model.bn
    
    def forward(self, x):
        features = self.backbone(x)
        features = self.dropout(features)
        embeddings = self.embedding_layer(features)
        embeddings = self.bn(embeddings)
        embeddings = F.normalize(embeddings, p=2, dim=1)
        return embeddings


# Create embedding model
embedding_model = EmbeddingModel(model).to(DEVICE).eval()

# Trace the model
example_input = torch.randn(1, 3, 160, 160).to(DEVICE)
traced_model = torch.jit.trace(embedding_model, example_input)

# Save
traced_model.save(SAVE_DIR / "face_embedding_model.pt")
print("✅ Saved TorchScript model: face_embedding_model.pt")

In [ ]:
# Export to ONNX for cross-platform deployment
import torch.onnx

torch.onnx.export(
    embedding_model,
    example_input,
    SAVE_DIR / "face_embedding_model.onnx",
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['embedding'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'embedding': {0: 'batch_size'}
    }
)

print("✅ Saved ONNX model: face_embedding_model.onnx")

In [ ]:
# Save label encoder and metadata
import json

metadata = {
    "model_version": "1.0.0",
    "embedding_size": model.embedding_size,
    "num_classes": model.num_classes,
    "input_size": [3, 160, 160],
    "normalization": {
        "mean": [0.5, 0.5, 0.5],
        "std": [0.5, 0.5, 0.5]
    },
    "classes": checkpoint['label_encoder_classes'],
    "training": {
        "best_epoch": checkpoint['epoch'],
        "val_accuracy": checkpoint['val_accuracy']
    }
}

with open(SAVE_DIR / "model_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Saved model metadata")

## 10. Inference Example

In [ ]:
def get_face_embedding(image_path: str, model, mtcnn, device, transform) -> Optional[np.ndarray]:
    """Get face embedding from image."""
    # Detect face
    img = Image.open(image_path).convert('RGB')
    face = mtcnn(img)
    
    if face is None:
        return None
    
    # Preprocess
    face_np = (face.permute(1, 2, 0).cpu().numpy() * 128 + 127.5).astype(np.uint8)
    augmented = transform(image=face_np)
    face_tensor = augmented['image'].unsqueeze(0).to(device)
    
    # Get embedding
    with torch.no_grad():
        embedding = model.get_embedding(face_tensor)
    
    return embedding.cpu().numpy()[0]


def compute_similarity(emb1: np.ndarray, emb2: np.ndarray) -> float:
    """Compute cosine similarity between two embeddings."""
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))


# Example: Compare two faces
# emb1 = get_face_embedding("path/to/face1.jpg", model, mtcnn, DEVICE, val_transform)
# emb2 = get_face_embedding("path/to/face2.jpg", model, mtcnn, DEVICE, val_transform)
# similarity = compute_similarity(emb1, emb2)
# print(f"Similarity: {similarity:.4f}")
# print(f"Same person: {similarity > 0.6}")

print("📥 Inference functions ready!")

## 11. Download Trained Models

In [ ]:
# Create zip of all outputs
import shutil

# Zip the checkpoints directory
shutil.make_archive("trained_models", 'zip', SAVE_DIR)

# Download
from google.colab import files
files.download("trained_models.zip")

print("\n📥 Download complete! Contents:")
print("   - best_model.pth (PyTorch checkpoint)")
print("   - face_embedding_model.pt (TorchScript)")
print("   - face_embedding_model.onnx (ONNX)")
print("   - model_metadata.json")
print("   - training_history.png")
print("   - confusion_matrix.png")
print("   - tsne_embeddings.png")

---

## 🎉 Training Complete!

### Next Steps:
1. **Download the trained models** using the cell above
2. **Copy to your CCTV system**: Place models in `models/` directory
3. **Update config.yaml** to point to the new model
4. **Build FAISS index** from criminal database embeddings

### Integration with CCTV System:

```python
# In your CCTV system's recognition module:
from facenet_pytorch import InceptionResnetV1
import torch

# Load custom trained model
model = torch.jit.load('models/face_embedding_model.pt')
model.eval()

# Get embedding for new face
embedding = model(face_tensor)

# Compare with criminal database using FAISS
distances, indices = faiss_index.search(embedding.numpy(), k=5)
```

---